# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [2]:
import pandas as pd

# Load and examine the recipes data
recipes = pd.read_csv('recipes.tsv', sep='\t')
print("Recipes dataset info:")
print(f"Shape: {recipes.shape}")
print(f"Columns: {list(recipes.columns)}")
print("\nFirst 10 recipes:")
print(recipes.head(10))
print("\nSample of recipe names and descriptions:")
for i in range(20):
    print(f"{recipes.iloc[i]['recipe_slug']}: {recipes.iloc[i].get('title', 'N/A')} - {recipes.iloc[i].get('description', 'N/A')[:100]}...")

Recipes dataset info:
Shape: (100, 3)
Columns: ['recipe_slug', 'recipe_title', 'recipe_introduction']

First 10 recipes:
                    recipe_slug                  recipe_title  \
0                       falafel                       Falafel   
1                    spamburger                    Spamburger   
2              bacon-fried-rice              Bacon Fried Rice   
3               chicken-fingers               Chicken Fingers   
4                   apple-crisp                   Apple Crisp   
5         cranberry-apple-crisp         Cranberry Apple Crisp   
6  bacon-chocolate-chip-cookies  Bacon Chocolate Chip Cookies   
7                        sujebi                        Sujebi   
8               pasta-primavera               Pasta Primavera   
9                         ramen                         Ramen   

                                 recipe_introduction  
0  Falafel is a popular Middle Eastern dish made ...  
1  Spamburger is a type of hamburger that is made... 

In [33]:
recipes = pd.read_csv('recipes.tsv', sep='\t')

def assign_raw_rating(row):
    slug = row['recipe_slug']
    if 'chocolate' in slug or 'brownies' in slug or 'bread' in slug:
        return 5
    elif 'bacon' in slug or 'spam' in slug:
        return 1
    elif 'crisp' in slug or 'cookie' in slug:
        return 4
    elif any(x in slug for x in ['ramen', 'chicken', 'falafel', 'pasta', 'sujebi']):
        return 2
    else:
        return 3

recipes['rating_raw'] = recipes.apply(assign_raw_rating, axis=1)
recipes['rating'] = (recipes['rating_raw'] - 1) * 0.25

ratings = recipes[['recipe_slug', 'rating']]
ratings.to_csv('ratings.tsv', sep='\t', index=False)


In [34]:
import pandas as pd

# Load your ratings file
ratings = pd.read_csv('ratings.tsv', sep='\t')

# Count how many recipes with rating 0.0
count_zero = (ratings['rating'] == 0.0).sum()

# Count how many recipes with rating 1.0
count_one = (ratings['rating'] == 1.0).sum()

print(f'Number of ratings == 0.0: {count_zero}')
print(f'Number of ratings == 1.0: {count_one}')


Number of ratings == 0.0: 14
Number of ratings == 1.0: 10


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [35]:
ratings = pd.read_csv('ratings.tsv', sep='\t')
recipe_tags = pd.read_csv('recipe-tags.tsv', sep='\t')

# All unique tags
all_tags = sorted(recipe_tags['recipe_tag'].unique())

# For each recipe, what tags does it have?
tagged = recipe_tags.groupby('recipe_slug')['recipe_tag'].apply(list).to_dict()

# Build feature rows
rows = []
for slug in ratings['recipe_slug']:
    features = {'recipe_slug': slug, 'bias': 1}
    recipe_tags_list = tagged.get(slug, [])
    for tag in all_tags:
        features[tag] = int(tag in recipe_tags_list)
    rows.append(features)

# DataFrame and save
features_df = pd.DataFrame(rows)
features_df.to_csv('features.tsv', sep='\t', index=False)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [36]:
# YOUR CHANGES HERE
from sklearn.linear_model import Ridge

features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

data = features.merge(ratings[['recipe_slug', 'rating']], on='recipe_slug')
X = data.drop(['recipe_slug', 'rating'], axis=1)
y = data['rating']

model = Ridge(alpha=1, fit_intercept=False)
model.fit(X, y)



,alpha,1
,fit_intercept,False
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [37]:
# YOUR CHANGES HERE
coefs = pd.DataFrame({
    'recipe_tag': X.columns,
    'coefficient': model.coef_
})
coefs.to_csv("model.tsv", sep="\t", index=False)


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [38]:
features = pd.read_csv('features.tsv', sep='\t')
coefs = pd.read_csv('model.tsv', sep='\t')

X = features.drop('recipe_slug', axis=1)
coef_vect = coefs.set_index('recipe_tag')['coefficient'].reindex(X.columns).values

scores = X.dot(coef_vect)

result = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_estimate': scores
})
result.to_csv('estimates.tsv', sep='\t', index=False)



Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [39]:
# YOUR CHANGES HERE

import numpy as np

features = pd.read_csv('features.tsv', sep='\t')
coefs = pd.read_csv('model.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

X = features.drop('recipe_slug', axis=1).values
alpha = 1
A = X.T @ X + alpha * np.identity(X.shape[1])
A_inv = np.linalg.inv(A)
coef_vect = coefs.set_index('recipe_tag')['coefficient'].reindex(features.columns[1:]).values

score_estimate = X @ coef_vect
variance = np.sqrt(np.sum((X @ A_inv) * X, axis=1))
score_bound = score_estimate + 2 * variance

bounds = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_bound': score_bound
})
bounds.to_csv('bounds.tsv', sep='\t', index=False)


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [40]:
# YOUR CHANGES HERE

recommendations = []

remaining = set(features['recipe_slug'])
history_X = []
history_y = []

for _ in range(100):
    bounds = []
    for i, slug in enumerate(features['recipe_slug']):
        if slug not in remaining:
            continue
        x = features.drop('recipe_slug', axis=1).iloc[i].values
        if history_X:
            X_hist = np.vstack(history_X)
            A = X_hist.T @ X_hist + alpha * np.identity(len(x))
            A_inv = np.linalg.inv(A)
            coef = np.linalg.solve(A, X_hist.T @ np.array(history_y))
            estimate = x @ coef
            var = np.sqrt(x @ A_inv @ x)
        else:
            estimate = 0
            var = np.sqrt(alpha * np.sum(x ** 2))
        bound = estimate + 2 * var
        bounds.append((slug, bound, estimate, x))
    top = max(bounds, key=lambda t: t[1])
    reward = float(ratings.loc[ratings['recipe_slug'] == top[0], 'rating'])
    recommendations.append({'recipe_slug': top[0], 'score_bound': top[1], 'reward': reward})
    history_X.append(top[3])
    history_y.append(reward)
    remaining.remove(top[0])

pd.DataFrame(recommendations).to_csv('recommendations.tsv', sep='\t', index=False)


/tmp/ipykernel_16498/3596101860.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  reward = float(ratings.loc[ratings['recipe_slug'] == top[0], 'rating'])
/tmp/ipykernel_16498/3596101860.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  reward = float(ratings.loc[ratings['recipe_slug'] == top[0], 'rating'])
/tmp/ipykernel_16498/3596101860.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  reward = float(ratings.loc[ratings['recipe_slug'] == top[0], 'rating'])
/tmp/ipykernel_16498/3596101860.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  reward = float(ratings.loc[ratings['recipe_slug'

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.